In [1]:
# import torch
# torchversion = torch.__version__

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [2]:
# from google.colab import drive
# drive.mount("/content/gdrive")
# %cd /content/gdrive/MyDrive/QSeer/

# Load Packages

In [ ]:
import joblib
import numpy as np
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, InMemoryDataset
import torch

import os
import networkx as nx
import tensorcircuit as tc
import tensorflow as tf
import random
from tensorflow.keras import backend as K


import torch.nn as nn
from torch.nn import Linear, Sequential, ReLU#, Dropout, BatchNorm1d
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GINConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Batch, Dataset

import torch.optim as optim
from torch.optim import lr_scheduler

print(torch.cuda.is_available())
K = tc.set_backend("numpy") # solve JIT failed issue
tc.set_contractor("auto")


True


functools.partial(<function custom at 0x75b546083eb0>, optimizer=<function auto at 0x75b5d4155870>, memory_limit=None, debug_level=0)

# Create Dataset

The train data got error, so we just extract and process half the data from now on

In [ ]:
def create_subset(file_path, ratio=0.5):
    """
    Reads a dataset file, takes a portion of the data items according to ratio,
    and saves to a new file with _{int(ratio*100)} suffix.
    """
    # Load the full dataset
    data_list = joblib.load(file_path)
    
    if not data_list:
        print("No data found in the file.")
        return

    # Calculate number of items to keep
    n_keep = max(1, int(len(data_list) * ratio))
    subset_list = data_list[:n_keep]

    # Create new file name
    dir_name, base_name = os.path.split(file_path)
    name, ext = os.path.splitext(base_name)
    new_file_name = os.path.join(dir_name, f"{name}_{int(ratio*100)}{ext}")

    # Save the subset
    joblib.dump(subset_list, new_file_name)
    print(f"Saved subset ({len(subset_list)} items) to {new_file_name}")

# Example usage
# create_subset('data/graph_train.pkl', ratio=0.5)
# create_subset('data/graph_test.pkl', ratio=0.5)

In [ ]:
def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

class GraphDataset(InMemoryDataset):
  def __init__(self, root, file_name=None, transform=None, pre_transform=None, pre_filter=None):
    self.filename = file_name
    super(GraphDataset, self).__init__(root, transform, pre_transform, pre_filter)
    self.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return [self.filename]

  @property
  def processed_file_names(self):
    return [self.filename + ".pt"]

  def process(self):
      data_list = []
      my_list = joblib.load(self.filename)

      for item in my_list:
          graph = item["graph"]
          gamma = np.array(item["gamma"], dtype=np.float32)
          beta = np.array(item["beta"], dtype=np.float32)

          # Layer
          layer_val = np.shape(gamma)[0]
          n_values = 4
          layer = np.eye(n_values)[layer_val - 1]

          # Label
          label = interleave_arrays(gamma, beta)
          label = np.pad(label, (0, 8 - label.shape[0]), mode='constant')

          # Edge index
          edge_index = torch.tensor(list(graph.edges())).t().contiguous()

          # Edge attr and node features
          edge_attr = torch.arange(graph.number_of_edges(), dtype=torch.float32).unsqueeze(1)
          x = torch.arange(graph.number_of_nodes(), dtype=torch.float32).unsqueeze(1)

          # Labels and layers as tensors
          y = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
          layer_tensor = torch.tensor(layer, dtype=torch.float32).unsqueeze(0)

          data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x, y=y, layer=layer_tensor)
          data_list.append(data)

      self.save(data_list, self.processed_paths[0])

# train_dataset = GraphDataset(root='./data/', file_name='./data/graph_train_50.pkl')
# test_dataset = GraphDataset(root='./data/', file_name='./data/graph_test.pkl')
# valid_dataset = GraphDataset(root='./data/', file_name='./data/graph_valid.pkl')

# print(train_dataset.num_node_features)
# print(train_dataset.num_classes)
# print(train_dataset[0].layer)

Check sample data

In [ ]:
my_list = joblib.load("data/graph_test.pkl")

In [ ]:
print("=== First 3 items in dataset ===")
for i, item in enumerate(my_list[:3]):
    print(f"\nItem {i}:")
    print("  graph:", item["graph"])
    print("  gamma:", item["gamma"])
    print("  beta :", item["beta"])
    print("  c    :", item["c"])
    print("  ci   :", item["ci"])

# Work on the first graph
graph = my_list[1]["graph"]
print("\n=== Selected Graph (Item 0) ===")
print("Number of nodes:", graph.number_of_nodes())
print("Number of edges:", graph.number_of_edges())
print("Edges:", list(graph.edges()))

# Edge index
edge_index = torch.tensor(list(graph.edges())).t().contiguous()
print("\nEdge Index Tensor:")
print(edge_index)
print("Shape:", edge_index.shape)

# Edge attr
edge_attr = torch.arange(graph.number_of_edges(), dtype=torch.float32).unsqueeze(1)
print("\nEdge Attr Tensor:")
print(edge_attr)
print("Shape:", edge_attr.shape)

# Node features
x = torch.arange(graph.number_of_nodes(), dtype=torch.float32).unsqueeze(1)
print("\nNode Feature Tensor (x):")
print(x)
print("Shape:", x.shape)


In [ ]:
def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

class GraphWDataset(InMemoryDataset):
  def __init__(self, root, file_name=None, transform=None, pre_transform=None, pre_filter=None):
    self.filename = file_name
    super(GraphWDataset, self).__init__(root, transform, pre_transform, pre_filter)
    self.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return [self.filename]

  @property
  def processed_file_names(self):
    return [self.filename + ".pt"]

  def process(self):
    data_list = []
    my_list = joblib.load(self.filename)
    index = 0
    for item in my_list:
      source_list = item["slist"]
      target_list = item["tlist"]

      gamma = np.array(item["gamma"], dtype=np.float32)
      beta =  np.array(item["beta"], dtype=np.float32)
      layer = np.array(np.shape(gamma)[0], dtype=np.int32)
      n_values = np.max(4)
      layer = np.eye(n_values)[layer-1]
      label = interleave_arrays(gamma, beta)
      label = np.pad(label, (0, 8 - np.shape(label)[0]), mode='constant', constant_values=0)

      edge_index = torch.tensor([source_list, target_list], dtype=torch.int64)
      edge_attr = torch.tensor([[item["wlist"][i]] for i in range(len(item["wlist"]))], dtype=torch.float32)
      num_nodes = torch.max(edge_index).item() + 1
      x = torch.tensor([[i] for i in range(num_nodes)], dtype=torch.float32)
      y = torch.tensor(label, dtype=torch.float32).flatten()
      y = torch.unsqueeze(y, 0)
      layer = torch.tensor(layer, dtype=torch.float32).flatten()
      layer = torch.unsqueeze(layer, 0)
      data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x, y=y, layer=layer)
      data_list.append(data)

    self.save(data_list, self.processed_paths[0])


# train_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_train.pkl')
# test_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_test.pkl')
# valid_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_valid.pkl')

# print(train_dataset.num_node_features)
# print(train_dataset.num_classes)
# print(train_dataset[0].layer)

In [ ]:

def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

class GraphGDataset(InMemoryDataset):
  def __init__(self, root, file_name=None, num_layer = 4, transform=None, pre_transform=None, pre_filter=None):
    self.filename = file_name
    self.num_layer = num_layer
    super(GraphGDataset, self).__init__(root, transform, pre_transform, pre_filter)
    self.load(self.processed_paths[0])

  @property
  def raw_file_names(self):
    return [self.filename]

  @property
  def processed_file_names(self):
    return [self.filename + "_" + str(self.num_layer) + "_.pt"]

  def process(self):
    data_list = []
    graph_list = joblib.load(self.filename)
    index = 0
    for graph in graph_list:
      source_list = None
      target_list = None
      edge_list = graph.edges()
      for edge in edge_list:
        if source_list is None:
          source_list = np.array([edge[0]])
          target_list = np.array([edge[1]])
        else:
          source_list = np.append(source_list, edge[0])
          target_list = np.append(target_list, edge[1])

      layer = np.array(self.num_layer, dtype=np.int32)
      n_values = np.max(4)
      layer = np.eye(n_values)[layer-1]

      edge_index = torch.tensor([source_list, target_list], dtype=torch.int64)
      edge_attr = torch.tensor([[i] for i in range(len(edge_list))], dtype=torch.float32)

      x = torch.tensor([[i] for i in range(graph.number_of_nodes())], dtype=torch.float32)
      layer = torch.tensor(layer, dtype=torch.float32).flatten()
      layer = torch.unsqueeze(layer, 0)
      data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x, y=None, layer=layer)
      data_list.append(data)

    self.save(data_list, self.processed_paths[0])

# Create Model

In [ ]:
class DGNN(torch.nn.Module):
  def __init__(self, n_x, n_y, dim_h):
    super(DGNN, self).__init__()
    hidden_channels_gcn = dim_h * 2
    hidden_channels_gat = hidden_channels_gcn * 2
    hidden_channels_gin = hidden_channels_gat * 2

    # Initialize the first GCNConv layer in the forward method
    self.conv1 = GCNConv(n_x, hidden_channels_gcn)
    self.hidden_channels_gcn = hidden_channels_gcn
    self.conv2 = GCNConv(hidden_channels_gcn, hidden_channels_gcn)
    self.gat_conv1 = GATConv(hidden_channels_gcn, hidden_channels_gat)
    self.gat_conv2 = GATConv(hidden_channels_gat, hidden_channels_gat)

    mlp = torch.nn.Sequential(
        Linear(hidden_channels_gat, hidden_channels_gin),
        ReLU(),
        Linear(hidden_channels_gin, hidden_channels_gin)
    )
    self.gin_conv1 = GINConv(mlp)
    self.out = Linear(hidden_channels_gin + 4, hidden_channels_gin)
    self.out2 = Linear(hidden_channels_gin, n_y)

  def forward(self, x, edge_index, edge_weight, layer, batch):
    x = F.relu(self.conv1(x = x, edge_index = edge_index, edge_weight = edge_weight))
    #print(x)
    x = F.relu(self.conv2(x = x, edge_index = edge_index, edge_weight = edge_weight))
    x = F.relu(self.gat_conv1(x = x, edge_index = edge_index, edge_attr = edge_weight))
    x = F.relu(self.gat_conv2(x = x, edge_index = edge_index, edge_attr = edge_weight))
    x = F.relu(self.gin_conv1(x = x, edge_index = edge_index))
    x = global_mean_pool(x, batch)
    #print(x.shape)
    #print(layer.shape)
    x = torch.cat([x, layer], dim=1)
    x = self.out(x)
    x = self.out2(x)
    return x

# Train

In [ ]:
train_dataset = GraphDataset(root='./data/', file_name='./data/graph_train_50.pkl')
test_dataset = GraphDataset(root='./data/', file_name='./data/graph_test.pkl')
valid_dataset = GraphDataset(root='./data/', file_name='./data/graph_valid.pkl')

model = DGNN(n_x=train_dataset.num_node_features, n_y=train_dataset.num_classes, dim_h=256).to("cuda")
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

n_epochs = 3
batch_size = 4096
myloader = DataLoader(train_dataset, batch_size=batch_size)
youloader = DataLoader(valid_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)

best_val_loss = float('inf')
save_path = './best_model.pth'

for epoch in range(n_epochs):
  model.train()
  mse_list = []
  for data in myloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    mse_list.append(float(loss))
  train_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} training mse {train_mse:.4f}")

  model.eval()
  mse_list = []
  for data in youloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    mse_list.append(float(loss))
  valid_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} valid mse {valid_mse:.4f}")

  if valid_mse < best_val_loss:
    best_val_loss = valid_mse
    torch.save(model.state_dict(), save_path)
    print(f'Saved best model with validation loss: {best_val_loss:.4f}')

  scheduler.step()

In [ ]:
train_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_train.pkl')
test_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_test.pkl')
valid_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_valid.pkl')

model = DGNN(n_x=train_dataset.num_node_features, n_y=train_dataset.num_classes, dim_h=256).to("cuda")
#model.load_state_dict(torch.load(save_path, weights_only=True))

loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

n_epochs = 3
batch_size = 1024
myloader = DataLoader(train_dataset, batch_size=batch_size)
youloader = DataLoader(valid_dataset, batch_size=batch_size)
testloader = DataLoader(test_dataset, batch_size=batch_size)

best_val_loss = float('inf')
save_path = './best_weighted_model.pth'

for epoch in range(n_epochs):
  model.train()
  mse_list = []
  for data in myloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    mse_list.append(float(loss))
  train_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} training mse {train_mse:.4f}")

  model.eval()
  mse_list = []
  for data in youloader:
    data = data.to("cuda")
    out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
    loss = loss_fn(out, data.y)
    mse_list.append(float(loss))
  valid_mse = sum(mse_list) / len(mse_list)
  print(f"model epoch {epoch} valid mse {valid_mse:.4f}")

  if valid_mse < best_val_loss:
    best_val_loss = valid_mse
    torch.save(model.state_dict(), save_path)
    print(f'Saved best model with validation loss: {best_val_loss:.4f}')

  scheduler.step()

# Test

In [ ]:
save_path = './best_model.pth'
test_dataset = GraphDataset(root='./data/', file_name='./data/graph_test.pkl')
model = DGNN(n_x=test_dataset.num_node_features, n_y=test_dataset.num_classes, dim_h=256).to("cuda")
model.load_state_dict(torch.load(save_path, weights_only=True))
model.eval()
testloader = DataLoader(test_dataset, batch_size=4096)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())

test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, "./test_result.pkl")

In [ ]:
save_path = './best_weighted_model.pth'
test_dataset = GraphWDataset(root='./data/', file_name='./data/weighted_graph_test.pkl')
model = DGNN(n_x=test_dataset.num_node_features, n_y=test_dataset.num_classes, dim_h=256).to("cuda")
model.load_state_dict(torch.load(save_path, weights_only=True))
model.eval()
testloader = DataLoader(test_dataset, batch_size=4096)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())

test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, "./test_result_weighted.pkl")

In [ ]:
save_path = './best_model.pth'
test_dataset1 = GraphGDataset(root='./data/', file_name='./data/graph_unseen_test.pkl', num_layer=1)
test_dataset2 = GraphGDataset(root='./data/', file_name='./data/graph_unseen_test.pkl', num_layer=2)
test_dataset3 = GraphGDataset(root='./data/', file_name='./data/graph_unseen_test.pkl', num_layer=3)


model = DGNN(n_x=1, n_y=8, dim_h=256).to("cuda")
model.load_state_dict(torch.load(save_path, weights_only=True))
model.eval()

testloader = DataLoader(test_dataset1, batch_size=250)

print(len(testloader))

test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())
test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, f"./unseen_result_1.pkl")


testloader = DataLoader(test_dataset2, batch_size=250)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())
test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, f"./unseen_result_2.pkl")

testloader = DataLoader(test_dataset3, batch_size=250)
test_result = []
for data in testloader:
  data = data.to("cuda")
  out = model(data.x, data.edge_index, data.edge_attr, data.layer, data.batch)
  test_result.append(out.cpu().detach().numpy())
test_result = np.concatenate(test_result, axis=0)
print(np.shape(test_result))
joblib.dump(test_result, f"./unseen_result_3.pkl")

# Quantum comparison

In [30]:
def interleave_arrays(arr1, arr2):
  return np.ravel(np.column_stack((arr1, arr2)))

def beta_to_qaoa(beta):
  return np.pi * np.array(beta, dtype=np.float32)

def gamma_to_qaoa(gamma):
  return -np.pi * np.array(gamma, dtype=np.float32) / 2

def all_to_qaoa(param, layer):
  pi0 = [np.pi] * layer
  pi1 = [-np.pi/2] * layer
  pi2 = interleave_arrays(pi1, pi0)
  result = pi2 * param
  return result.astype(np.float32)

def QAOAansatz(params, g=None, L=4, return_circuit=False):
    n_qubits = len(g.nodes)
    circuit = tc.Circuit(n_qubits)
    for qubit in range(n_qubits):
        circuit.H(qubit)

    for layer_index in range(L):
        for edge in g.edges:
            circuit.exp1(edge[0], edge[1], unitary=tc.gates._zz_matrix, theta=params[2 * layer_index])
        for qubit in range(n_qubits):
            circuit.rx(qubit, theta=params[2 * layer_index + 1])

    if return_circuit:
        return circuit

    energy = 0.0
    for edge in g.edges:
        energy += circuit.expectation_ps(z=[edge[0], edge[1]])

    return K.real(energy)

In [34]:

# Load data
graph_test = joblib.load("./data/graph_test.pkl")
result_test = joblib.load("./test_result.pkl")

# SAMPLING PARAMETERS
SAMPLE_SIZE = 50
BATCH_SIZE = 10

print(f"Total graphs: {len(graph_test)}")
print(f"Testing on: {SAMPLE_SIZE} graphs")
print(f"Batch size: {BATCH_SIZE}")

graph_index = 0
prediction_better_than_random = 0
prediction_better_than_ground_truth = 0

for batch_start in range(0, min(SAMPLE_SIZE, len(graph_test)), BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, SAMPLE_SIZE, len(graph_test))
    print(f"\nProcessing batch: {batch_start} to {batch_end}")

    for i in range(batch_start, batch_end):
        graph_data = graph_test[i]
        g = graph_data["graph"]
        gamma_angles = gamma_to_qaoa(graph_data["gamma"])
        beta_angles = beta_to_qaoa(graph_data["beta"])
        num_layers = np.shape(gamma_angles)[0]

        # Prepare parameters
        ground_truth_params = interleave_arrays(gamma_angles, beta_angles)
        ground_truth_params = np.pad(ground_truth_params, (0, 8 - np.shape(ground_truth_params)[0]),
                                     mode='constant', constant_values=0)

        predicted_params = result_test[graph_index]
        predicted_params = all_to_qaoa(predicted_params, 4)

        # Three parameter sets
        random_params = np.random.randn(8) * 0.1
        parameter_sets = np.stack([random_params, ground_truth_params, predicted_params], axis=0)

        # Evaluate energies
        energies = []
        for circuit_index in range(len(parameter_sets)):
            energy = QAOAansatz(params=parameter_sets[circuit_index], g=g, L=num_layers)
            energies.append(float(energy))

        # Compare results
        if energies[2] < energies[0]:
            prediction_better_than_random += 1
        if energies[2] < energies[1]:
            prediction_better_than_ground_truth += 1

        graph_index += 1

        if graph_index % 10 == 0:
            print(f"Processed: {graph_index}/{SAMPLE_SIZE}")
            print(f"  Better than random: {prediction_better_than_random}/{graph_index} "
                  f"({100 * prediction_better_than_random / graph_index:.1f}%)")
            print(f"  Better than ground truth: {prediction_better_than_ground_truth}/{graph_index} "
                  f"({100 * prediction_better_than_ground_truth / graph_index:.1f}%)")

# Final results
print("\n" + "=" * 50)
print("FINAL RESULTS")
print("=" * 50)
print(f"Total tested: {graph_index}")
print(f"Better than random: {prediction_better_than_random} "
      f"({100 * prediction_better_than_random / graph_index:.1f}%)")
print(f"Better than ground truth: {prediction_better_than_ground_truth} "
      f"({100 * prediction_better_than_ground_truth / graph_index:.1f}%)")

Total graphs: 81958
Testing on: 50 graphs
Batch size: 10

Processing batch: 0 to 10
Processed: 10/50
  Better than random: 2/10 (20.0%)
  Better than ground truth: 2/10 (20.0%)

Processing batch: 10 to 20
Processed: 20/50
  Better than random: 5/20 (25.0%)
  Better than ground truth: 2/20 (10.0%)

Processing batch: 20 to 30
Processed: 30/50
  Better than random: 5/30 (16.7%)
  Better than ground truth: 3/30 (10.0%)

Processing batch: 30 to 40
Processed: 40/50
  Better than random: 7/40 (17.5%)
  Better than ground truth: 4/40 (10.0%)

Processing batch: 40 to 50
Processed: 50/50
  Better than random: 8/50 (16.0%)
  Better than ground truth: 4/50 (8.0%)

FINAL RESULTS
Total tested: 50
Better than random: 8 (16.0%)
Better than ground truth: 4 (8.0%)


In [33]:
energies

[-1.2394050732254982, -2.0816769301891327, 0.9993368089199066]

In [ ]:

# QAOA_vvag = K.jit(
#   tc.backend.vvag(QAOAansatz, argnums=0, vectorized_argnums=0), static_argnums=(1, 3)
# )
# opt = K.optimizer(tf.keras.optimizers.Adam(1e-3))

# graph_test = joblib.load("./data/graph_test.pkl")
# result_test = joblib.load("./test_result.pkl")

# index = 0
# better = 0
# best = 0

# for i, graph in enumerate(graph_test):
#   g = graph["graph"]
#   gamma = gamma_to_qaoa(graph["gamma"])
#   beta =  beta_to_qaoa(graph["beta"])
#   layer = np.shape(gamma)[0]
#   label = interleave_arrays(gamma, beta)
#   label = np.pad(label, (0, 8 - np.shape(label)[0]), mode='constant', constant_values=0)
#   label = np.expand_dims(label, axis=0)
#   para = result_test[index]
#   para = all_to_qaoa(para, 4)
#   para = np.expand_dims(para, axis=0)
#   p1 = K.implicit_randn(shape=[1, 8], stddev=0.1) # random
#   p2 = tf.convert_to_tensor(label) # ground truth
#   p3 = tf.convert_to_tensor(para) # predict
#   p = tf.concat([p1, p2, p3], 0)

#   LL = []
#   for num_circuit in range(3):
#     #c = QAOAansatz(params=p[num_circuit], g=prob, return_circuit=True)
#     loss = QAOAansatz(params=p[num_circuit], g=g, L=layer)
#     LL.append(K.numpy(loss))

#   if LL[2] < LL[0]:
#     better += 1

#   if LL[2] < LL[1]:
#     best += 1

#   index += 1
#   if index % 100 == 0:
#     print(index)
#     print(better)
#     print(best)


# print(index)
# print(better)
# print(best)